In [1]:
%matplotlib inline

from config import configs

import os
import io
import numpy as np
import matplotlib
import timeit
import random
import cv2
import deepdish as dd

import matplotlib.pyplot as plt

from pprint import pprint
from datetime import datetime
from pixor_targets import PIXORTargets
from skimage.transform import resize
from scipy.interpolate import griddata
from core.kitti import KITTI
from pixor_utils.model_utils import load_model, save_model
from data_utils.training_gen import TrainingGenerator
from data_utils.generator import Generator, KITTIGen
from tt import bev
from data_utils.generator import KITTIGen
from pixor_utils.post_processing import nms_bev
from test_utils.unittest import test_pc_encoder, test_target_encoder
from encoding_utils.pointcloud_encoder import OccupancyCuboidKITTI
from encoding_utils.voxelizer import BEVVoxelizer

In [2]:
DS_DIR = os.path.expanduser(configs['dataset_path'])

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = configs["gpu_id"]
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

In [3]:
# Point Cloud Encoder
INPUT_SHAPE = configs['input_shape']

# Training
BATCH_SIZE = configs['hyperparams']['batch_size']
LEARNING_RATE = configs['hyperparams']['lr']
EPOCHS = configs['hyperparams']['epochs']
NUM_THREADS = configs['hyperparams']['num_threads']
MAX_Q_SIZE = configs['hyperparams']['max_q_size']

In [4]:
kitti = KITTI(DS_DIR, configs['training_target'])

train_ids = kitti.get_ids('train')
val_ids = kitti.get_ids('val')
micro_ids = kitti.get_ids('micro')

In [5]:
print(len(train_ids) // 2)

1856


In [6]:
pc_encoder = OccupancyCuboidKITTI(0, 70, -40, 40, -1, 3, [0.1, 0.1, 0.4]) # 0.1, 0.1, 0.4
pc_encoder = BEVVoxelizer(n_x=448, n_y=512, n_z=32, side_range=(0,70), fwd_range=(-40,40), height_range=(-1, 3), filter_external=True)

In [7]:
target_encoder = PIXORTargets(shape=(128, 112), 
                              stats=dd.io.load('kitti_stats/stats.h5'),
                              P_WIDTH=70, P_HEIGHT=80, P_DEPTH=4, 
                              subsampling_factor=(0.75, 1.25))

In [8]:
train_gen = KITTIGen(kitti, train_ids, BATCH_SIZE, pc_encoder=pc_encoder, target_encoder=target_encoder)

In [9]:
for batch in train_gen:
#     start = timeit.default_timer()
    bev, rgb_img, depth_map, intensity_map, height_map, (obj, geo) = batch
#     print('took {}'.format(timeit.default_timer() - start))
#     print('bev.shape          ', bev.shape)
#     print('rgb_img.shape      ', rgb_img.shape)
#     print('depth_map.shape    ', depth_map.shape)
#     print('intensity_map.shape', intensity_map.shape)
#     print('height_map.shape   ', height_map.shape)
#     print('obj.shape          ', obj.shape)
#     print('geo.shape          ', geo.shape)
#     print('------------------------')

batch took 1.0157748167403042
batch took 0.915757458191365
batch took 0.8588674669153988
batch took 0.9169203108176589
batch took 1.040075909346342
batch took 0.9025516859255731
batch took 0.9003315637819469
batch took 0.8899878710508347
batch took 0.899451662786305
batch took 0.9381090039387345
batch took 0.8799267448484898
batch took 0.8637025728821754
batch took 0.9038570336997509
batch took 0.8554720152169466
batch took 0.8608772037550807
batch took 0.8924354091286659
batch took 0.9297924409620464
batch took 0.8629146139137447
batch took 0.8608400700613856
batch took 0.8616316118277609
batch took 0.9049537046812475
batch took 1.0509850909002125
batch took 0.855754854157567
batch took 0.8373381169512868
batch took 1.0046411762014031
batch took 0.8605670970864594
batch took 1.1310304342769086
batch took 0.906233505345881


KeyboardInterrupt: 